In [1]:
import keet_database as kdb
import cv2
import pandas as pd
import tensorflow as tf
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import numpy as np
import time 

In [6]:
############################################################################################################*

##########* LOAD THE MODEL ################################
path_save = 'C:\\Users\\hgera\\OneDrive\\Escritorio\\PY4E\\Keet\\Codes\\NN_all_statics\\all_statics_model.h5'
model = tf.keras.models.load_model(path_save)
scaler = joblib.load('C:\\Users\\hgera\\OneDrive\\Escritorio\\PY4E\\Keet\\Codes\\NN_all_statics\\scaler.pkl')
dict_labels = {0: 'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'L', 10:'M', 11:'N', 12:'O', 13:'P', 14:'R', 15:'S', 16:'T', 17:'U', 18:'V', 19:'W', 20:'Y'}
start_time = time.time()
delay_time = 0.5
predicted = False
#########* CAMERA SETTINGS ###########

cap, width, height = kdb.camera_settings(width_cam= 1280, height_cam= 720, camera=1) #* Width and height of the camera
                                                                                    #* 0 for the default camera, 1 for the external camera	 

#########* PARAMETERS ###########

imgformat, dataformat, tiempo_de_espera, ventana_de_tiempo = kdb.format(imgformat = 'jpg', dataformat= 'csv', waiting_time= 3, record_time = 2)

##########* Begin parameters ################# 

time_frames, t, t1, timeflag = kdb.time_set()

num_hand = kdb.n_hand()

mpHands, hands, mpDraw = kdb.hand_medipip(num_hand)

window_move = kdb.wind_move(roi1_x=0.1, roi1_y=0.4, roi2_x=0.1, roi2_y=0.6)

cTime, pTime, fps, Ts, time_frames = kdb.frame_settings()

#*FIRTS ARGUMENT FOR ROI 1 AND THE SECOND FOR ROI 2

###################* While loop for tracking    #################  

while True:
    
    current_time = time.time()
    
    ret, frame, frame_copy, frame_gray, frame_equali, results = kdb.read_frames(cap,hands,equali=True)
    #print(frame.shape)
    
    roi_save, save_len, point_save, lm_x_h1, lm_y_h1, lm_x_h2, lm_y_h2, lm_x_h1_roi, lm_y_h1_roi, lm_x_h2_roi, lm_y_h2_roi, flag = kdb.process_hand_landmarks(frame_equali= frame_equali,
                                                    results= results, width= width, height= height, t= t,tiempo_de_espera= tiempo_de_espera
                                                    ,save_len = None,print_lm=False, size_roi = 0.087, point_save={})       
    if current_time - start_time >= delay_time:  
        if flag == 1:
            data =  [ np.reshape(np.array(list(lm_x_h1.values())),(21,1))
                     , np.reshape(np.array(list(lm_x_h1_roi.values())),(21,1)), 
                      np.reshape(np.array(list(lm_y_h1.values())),(21,1)), 
                      np.reshape(np.array(list(lm_y_h1_roi.values())),(21,1)),]      
            data = np.concatenate(data,axis=1)
            data = data.reshape(1, 84)
            data_normalized = scaler.transform(data)
            predictions = model.predict(data_normalized, verbose=1)
            predicted_class = np.argmax(predictions, axis=1)
            # print(f'Predicción: {dict_labels[predicted_class[0]]}') 
            predicted = True
        start_time = current_time      
    if predicted == True and point_save != {}:
        cv2.putText(frame,dict_labels[predicted_class[0]], (point_save['h1_x_max'], point_save['h1_y_min']-40), cv2.FONT_HERSHEY_SIMPLEX, 2, (50, 50, 200), 3)
         
    #*######### ENDS IF ##############
        
    cTime, fps, Ts, pTime, time_frames = kdb.ends_if(cTime, fps, Ts, pTime, time_frames)
        

    ################* DRAW RECTANGULOS and text ###############

    kdb.draw_text_and_rectangles(point_save, frame, width, height, fps, draw_rectangules=True,draw_text=True)

    ##############* SHOW THE FRAMES #############

    SAVED = kdb.main_show(frame = frame, SAVED = None, width= width, height=height, roi_save= roi_save, window_move= window_move, df = None, RECORDING = None, t1 = None, save_len = None)
        
    #####* RESET THE LIST ########

    roi_save, point_save= kdb.reset_save(roi_save)

    ###* EXIT
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break   #press q for exit
cap.release()
cv2.destroyAllWindows()


Resolution: 1280,720
using external camera
Format: jpg,csv
Waiting time: 3 seconds, Recording time: 2 seconds
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━

(21,)

In [35]:
import platform

# Obtener el sistema operativo
os_name = platform.system()
os_name

'Windows'

In [36]:
import time
import numpy as np
time1 = time.time()
lista = []
for i in range(10000000):
    lista.append(i)
time2 = time.time()
print(time2-time1)

1.359290361404419


In [37]:
time1 = time.time()
lista = np.zeros(10000000)
for i in range(10000000):
    lista[i] = i
time2 = time.time()
print(time2-time1)


2.1223902702331543


In [38]:
import numpy as np
x = np.linspace(1, 21, 21)
x = x.reshape(21, 1)
x
y = np.array([1])


In [39]:
import pandas as pd
import numpy as np


In [40]:
import numpy as np
np.find(point_save.keys(),'h2_x_min')

AttributeError: module 'numpy' has no attribute 'find'

In [ ]:
point_save['h1_x_min'] = 1
point_save


{'h1_x_min': 1, 'h1_y_min': 193, 'h1_x_max': 1092, 'h1_y_max': 730}

In [ ]:
np.any(x)

True

In [ ]:
np.any(y)

True

In [ ]:

int(x[0])

1

In [ ]:
x = {'a' :5, 'b': 6}
max(x.values())

6

In [ ]:
import numpy as np
list(x.values())

[5, 6]